In [ ]:
import pandas as pd
from google.colab import files

# Paso 1: Subir el archivo Excel
print("Sube el archivo Excel:")
uploaded = files.upload()
archivo = list(uploaded.keys())[0]

# Paso 2: Leer el Excel desde la fila 10 visible (índice 9), encabezado real en la 11 (índice 10)
df = pd.read_excel(archivo, skiprows=8, header=1)

# Paso 3: Limpiar nombres de columnas
df.columns = df.columns.str.replace('\n', ' ').str.strip()

# Paso 4: Renombrar columnas clave para facilitar el trabajo
df = df.rename(columns={
    'Nombre Name': 'Nombre',
    'Lote Batch': 'Lote',
    'F.Sacrificio Slaughter D.': 'F_Sacrificio',
    'D.prod/desp Prod/Debond': 'D_Prod_Desp',
    'F.Caducidad Expiry Date': 'F_Caducidad'
})

# Paso 5: Limpiar y estandarizar la columna "Nombre"
df['Nombre'] = df['Nombre'].astype(str).str.strip().str.upper()

# Eliminar filas sin nombre válido
df = df[~df['Nombre'].isin(['', 'NAN', 'nan', 'NaN']) & df['Nombre'].notna()]

# Asegurar que los lotes se traten como texto y eliminar ".0" si Excel los guardó como flotantes
df['Lote'] = df['Lote'].astype(str).str.replace('.0$', '', regex=True)

# Paso 6: Convertir columnas de fecha al formato DD/MM/AAAA
for col in ['F_Sacrificio', 'D_Prod_Desp', 'F_Caducidad']:
    df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%d/%m/%Y')

# Paso 7: Agrupar por "Nombre" y obtener valores únicos con coma final
resultado = {}

for nombre, grupo in df.groupby("Nombre"):
    lotes = grupo["Lote"].dropna().unique()
    sacrificios = grupo["F_Sacrificio"].dropna().unique()
    despieces = grupo["D_Prod_Desp"].dropna().unique()
    caducidades = grupo["F_Caducidad"].dropna().unique()

    resultado[nombre] = {
        "Lote(s)": ', '.join(lotes) + ',' if len(lotes) else '',
        "F.Sacrificio": ', '.join(sacrificios) + ',' if len(sacrificios) else '',
        "D.prod/desp": ', '.join(despieces) + ',' if len(despieces) else '',
        "F.caducidad": ', '.join(caducidades) + ',' if len(caducidades) else ''
    }

# Paso 8: Mostrar resumen por consola
for nombre, datos in resultado.items():
    print(f"\nNombre: {nombre}")
    print(f"Lote(s): {datos['Lote(s)']}")
    print(f"F.Sacrificio: {datos['F.Sacrificio']}")
    print(f"D.prod/desp: {datos['D.prod/desp']}")
    print(f"F.caducidad: {datos['F.caducidad']}")

# Paso 9: Convertir el resultado a DataFrame
df_resultado = pd.DataFrame.from_dict(resultado, orient='index').reset_index()
df_resultado = df_resultado.rename(columns={"index": "Nombre"})

# Paso 10: Exportar el resumen a un archivo Excel
df_resultado.to_excel("resumen_por_nombre.xlsx", index=False)
files.download("resumen_por_nombre.xlsx")